In [ ]:
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.cluster.hierarchy as sch
from statsmodels.graphics.mosaicplot import mosaic

from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

warnings.filterwarnings('ignore')


In [ ]:

def plot_tsne_results(tsne_results, n_components, state, labels=None):
    """
    Plots the 2D t-SNE results.
    
    Parameters:
    - tsne_results: A 2D array or DataFrame containing the t-SNE results.
    - n_components: Number of components used in the t-SNE algorithm.
    - state: Random state used in the t-SNE algorithm.
    - labels: Labels for each data point for coloring. If None, all points are colored with the same color.
    """
    
    # Extracting the two dimensions from the results
    x = tsne_results[:, 0]
    y = tsne_results[:, 1]
    
    plt.figure(figsize=(8, 6))
    
    # If labels are provided, plot using different colors for each label
    if labels is not None:
        unique_labels = list(set(labels))
        for label in unique_labels:
            idx = labels == label
            plt.scatter(x[idx], y[idx], label=label, alpha=0.7)
        plt.legend()
    else:
        plt.scatter(x, y, alpha=0.7)
    
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.title(f't-SNE {n_components} Components - Random State {state}')
    plt.grid(True)
    plt.show()



In [ ]:
sinasc_df = pd.read_csv('D:/Disco/Data/datasus/SINASC/sinasc_nascimentos/sinasc_nascimentos.csv', sep=';')

### Explorando os dados...

In [ ]:
sinasc_df.sample(frac=.25, random_state=18).head(10)

In [ ]:
sinasc_df.info()

### Pré processamento
- Verifica todas as colunas e preenche com o valor padrao quando possivel
- Ajusta o tipo de dados

In [ ]:
cols = [
    'escolaridade_mae', 'raca_mae', 'estado_civil_mae', 'nascimento_mae', 
    'ocupacao', 'nro_consultas_prenatal', 'cesaria_antes_parto', 
    'duracao_gestacao', 'gravidez', 'local_nascimento', 'parto', 
    'trabalho_parto', 'apresentacao_crianca', 'possui_anomalia', 'sexo', 'raca'
]

In [ ]:
sinasc_df['data_nascimento'] = pd.to_datetime(sinasc_df['data_nascimento'], format='%m/%Y')
sinasc_df['idade_mae'] = (pd.to_datetime(sinasc_df['data_extracao']) - sinasc_df['data_nascimento']).dt.days / 365.25


In [ ]:
#ajusta o tipo de dado para categoria em cada coluna selecionada e também cria uma coluna numerica correspondente
colsv = []
for col in cols:
    sinasc_df[col].fillna('IGNORADO', inplace=True)
    sinasc_df[col] = sinasc_df[col].astype('category')
    le = LabelEncoder()
    sinasc_df[col + '_v'] = le.fit_transform(sinasc_df[col])
    colsv.append(col + '_v')

sinasc_df['cbo_mae'].fillna('IGNORADO', inplace=True)


#### Plot de alguns graficos para visualizar os dados

In [ ]:
plt.figure(figsize=(8, 3))
plt.hist(sinasc_df['idade_mae'], bins=20, edgecolor='black', color='lightblue')
plt.xlabel('Idade da mãe')
plt.ylabel('Frequência')
plt.show()

In [ ]:
escolaridade_counts = sinasc_df['escolaridade_mae'].value_counts()

plt.figure(figsize=(6, 3))
sns.barplot(x=escolaridade_counts.index, y=escolaridade_counts.values)
plt.xticks(rotation=45)
plt.title('Distribuição de Escolaridade das Mães')
plt.xlabel('Escolaridade')
plt.ylabel('Contagem')
plt.show()

In [ ]:
parto_counts = sinasc_df['parto'].value_counts()

plt.figure(figsize=(8, 3))
sns.barplot(x=parto_counts.index, y=parto_counts.values)
plt.title('Tipos de Parto')
plt.xlabel('Tipo de Parto')
plt.ylabel('Contagem')
plt.show()

In [ ]:
# tenta encontrar variaveis relacionadas
corr_matrix = sinasc_df.select_dtypes(include=[np.number]).corr()

plt.figure(figsize=(19, 7))
sns.heatmap(corr_matrix, annot=True, cmap='Blues')
plt.show()

- Na matriz de correlação é possível verificar algumas variáveis que podem ter relação, o que pode ajudar na clusterização
    - peso: duracao_gestacao, gravidez
    - escolaridade_mae: raca_mae, estado_civil_mae, nro_consultas_prenatal, parto
    - estado_civil_mae: nro_consultas_prenatal, cesaria_antes_parto, parto
    - nro_consultas_prenatal: cesaria_antes_parto, duracao_gestacao, parto
    - duracao_gestacao: gravidez, apresentacao_crianca

In [ ]:
threshold = 0.15  # Define o limiar de alta correlação
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            col_pair = (corr_matrix.columns[i], corr_matrix.columns[j])
            high_corr_pairs.append(col_pair)

#print(high_corr_pairs)

corr_cols = set()
for pair in high_corr_pairs:
    corr_cols.add(pair[0])
    corr_cols.add(pair[1])

corr_cols = list(corr_cols)
print(corr_cols)

### Clusterização

In [ ]:
pca = PCA(n_components=2)  # Ou outro número de componentes

scaler = StandardScaler()
data_numeric = sinasc_df[corr_cols]#.select_dtypes(include=[np.number])
scaled_data = scaler.fit_transform(data_numeric)
pca_result = pca.fit_transform(scaled_data)

# Adiciona os componentes principais ao DataFrame original para visualização
sinasc_df['PCA1'] = pca_result[:, 0]
sinasc_df['PCA2'] = pca_result[:, 1]

#print components column names
print(sinasc_df.columns[-2:])
#print components values
print(pca.components_)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(sinasc_df['PCA1'], sinasc_df['PCA2'], c=sinasc_df['PCA2'], cmap='plasma')
plt.xlabel('Primeiro Componente Principal')
plt.ylabel('Segundo Componente Principal')
plt.show()

In [ ]:
print(f'Variância explicada por cada componente: {pca.explained_variance_ratio_}')


In [ ]:
tsne = TSNE(n_components=3, init='pca', method='exact', random_state=18)
tsne_result = tsne.fit_transform(pd.DataFrame(scaled_data, columns=corr_cols).sample(frac=.25, random_state=18))

sinasc_df['TSNE1'] = tsne_result[:, 0]
sinasc_df['TSNE2'] = tsne_result[:, 1]


In [ ]:
plot_tsne_results(tsne_result, 3, 18)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(sinasc_df['TSNE1'], sinasc_df['TSNE2'], cmap='plasma')
plt.xlabel('Primeiro Componente TSNE')
plt.ylabel('Segundo Componente TSNE')
plt.show()

- KMEANS

In [ ]:
wcss = []
interval = range(1, 30)
for i in interval:  # Teste um intervalo de valores de K
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

plt.plot(interval, wcss)
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=10, init='k-means++', random_state=42)
cluster_labels = kmeans.fit_predict(scaled_data)
sinasc_df['Cluster_KMeans'] = cluster_labels

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=30)  
cluster_labels = dbscan.fit_predict(scaled_data)
sinasc_df['Cluster_DBSCAN'] = cluster_labels

In [ ]:
score = silhouette_score(scaled_data, cluster_labels)
print(f'Silhouette Score: {score}')